In [95]:
import dawdreamer as daw
import numpy 
from scipy.io import wavfile
from pathlib import Path

Load DAWDreamer engine

In [96]:
engine = daw.RenderEngine(44100, 512)

In [97]:
engine.set_bpm(120.)

In [98]:
synth = engine.make_plugin_processor("synth", "C:\\Program Files\\Common Files\\VST3\\TAL-NoiseMaker.vst3\\Contents\\x86_64-win\\TAL-NoiseMaker.vst3")

Check initial parameters of synth

In [99]:
synth.get_parameters_description()

[{'index': 0,
  'name': '-',
  'numSteps': 2147483647,
  'isBoolean': False,
  'isDiscrete': False,
  'label': '',
  'category': 'genericParameter',
  'text': '0.5000000',
  'currentValText': '0.5000000',
  'isMetaParameter': False,
  'isAutomatable': True,
  'defaultValue': 0.5,
  'defaultValueText': '0.5000000',
  'min': '0.0000000',
  'max': '1.0000000',
  'valueStrings': []},
 {'index': 1,
  'name': 'Master Volume',
  'numSteps': 2147483647,
  'isBoolean': False,
  'isDiscrete': False,
  'label': '',
  'category': 'genericParameter',
  'text': '0.4080000',
  'currentValText': '0.4080000',
  'isMetaParameter': False,
  'isAutomatable': True,
  'defaultValue': 0.4999000132083893,
  'defaultValueText': '0.4999000',
  'min': '0.0000000',
  'max': '1.0000000',
  'valueStrings': []},
 {'index': 2,
  'name': 'Filter Type',
  'numSteps': 2147483647,
  'isBoolean': False,
  'isDiscrete': False,
  'label': '',
  'category': 'genericParameter',
  'text': '0.0000000',
  'currentValText': '0.00

In [100]:
def generate_preset(synth):
    # generate random float between 0 and 1, 88 times
    params = [numpy.random.rand() for _ in range(88)]
    print(params)
    # Set the parameters
    for i, param in enumerate(params):
        synth.set_parameter(i, param)
    return params
    

In [ ]:
def export_sample(synth, preset_name):
    # (MIDI note, velocity, start sec, duration sec)
    synth.add_midi_note(60, 100, 0.0, 5.)
    engine.load_graph([
        (synth, [])
        ])
    engine.render(2.)
    audio = engine.get_audio()

    directory = Path('./samples/')
    directory.mkdir(parents=True, exist_ok=True)
    
    wavfile.write(directory / (preset_name + '.wav'), 44100, audio.transpose())

In [102]:
def export_preset(synth, num_to_generate=1):
    for i in range(num_to_generate):
        # Generate unique preset string
        preset_name = "preset_" + str(i)
        params = generate_preset(synth)

        # Save 'preset_name', and all params to one row in csv
        with open(f'presets.csv', 'a') as f:
            f.write(f'{preset_name},')
            for param in params:
                f.write(f'{param},')
            f.write('\n')
        
        # export sound
        export_sample(synth, preset_name)

In [103]:
export_preset(synth, 1)

[0.722775875019751, 0.7112535612220102, 0.6440045739554985, 0.18018767044008555, 0.18361321419164311, 0.6216355074405384, 0.9714999253591495, 0.49905459751659176, 0.620246427812481, 0.3259664886596213, 0.46317494703648976, 0.8025923025795162, 0.7326589377458778, 0.3499644026553905, 0.6444009229121779, 0.6794597807779187, 0.6723121897149165, 0.2262989221902123, 0.9213982083133936, 0.9578823082559613, 0.41856214679146897, 0.21337162522265252, 0.12708033493007187, 0.6650332107356125, 0.6712618627715723, 0.1461256457946638, 0.6570691321666927, 0.6048943213059145, 0.3309976213009166, 0.42653577137208465, 0.716951492216227, 0.38215073579283343, 0.8577173073564605, 0.2615696906652034, 0.8323192151467323, 0.8597195055838028, 0.00340782668553552, 0.11009622065469171, 0.7433265835230825, 0.11847586955445588, 0.2581808187523631, 0.5465058349879985, 0.4969128846849732, 0.26716270050381463, 0.2984457324582541, 0.10098400101654825, 0.7953258644282376, 0.3002532400515805, 0.3355365065070679, 0.056608

In [79]:
def load_preset(synth, preset_name):
    with open(f'presets.csv', 'r') as f:
        for line in f:
            if line.startswith(preset_name):
                params = line.split(',')[1:]
                for i, param in enumerate(params):
                    # if param is new line, skip
                    if param == '\n':
                        continue
                    synth.set_parameter(i, float(param))

try to load a preset, save it out, to see if it matches generated file

In [83]:
load_preset(synth, 'preset_3')
export_sample(synth, 'test')